In [1]:
import numpy as np
import pandas as pd
import sklearn
from sklearn import tree
import matplotlib.pyplot as plt
import warnings
from sklearn.model_selection import train_test_split
from scipy.stats import skew
from sklearn.tree import DecisionTreeClassifier

warnings.filterwarnings('ignore')

%matplotlib inline 

##read the cvs file into readable panda
dmtest=pd.read_csv("./dmtest.csv")
dmtrain=pd.read_csv("./dmtrain.csv")
print(dmtrain)
print(dmtest)

        id  n24  rev24  revlast  elpsdm  ordfreq  ordcat  response
0        1    2   30.0     20.0      24        1       6         1
1        2   25  207.0     20.0      24        1       6         0
2        3    5   52.0     15.0      24        1       6         0
3        4   11  105.0     15.0      24        1       6         0
4        5    2   32.0     17.0      24        1       6         0
...    ...  ...    ...      ...     ...      ...     ...       ...
1995  1996   12  105.0     20.0       1        2       6         0
1996  1997   36  786.0      5.0       1        2       6         0
1997  1998    7  415.0    100.0       1        1       7         0
1998  1999    5   77.0     25.0       1        1       7         0
1999  2000   34  469.0     31.0       1        1       6         0

[2000 rows x 8 columns]
        id  n24  rev24  revlast  elpsdm  ordfreq  ordcat  response
0     5001    5  186.0       18      24        1       6         1
1     5002    2   30.0       15      

In [2]:
## Question 1

colNames = list(dmtrain.columns)

## find how skewed each variable is
skewed_vars = dmtrain.skew(axis = 0, skipna = True)

## for all all the varibles skewed more than 1.5, use log transformation on them and replace the value
for x in range(len(skewed_vars)):
    skewedVal = skewed_vars[x]
    if skewedVal > 1.5:
        transformed = "transformed_" + colNames[x]
        dmtrain[transformed] = np.log(dmtrain[colNames[x]]+1)
        dmtrain.drop(colNames[x], axis = "columns", inplace = True)

dmtrain.head()

,id,elpsdm,ordfreq,ordcat,response,transformed_n24,transformed_rev24,transformed_revlast
0,1,24,1,6,1,1.098612,3.433987,3.044522
1,2,24,1,6,0,3.258097,5.337538,3.044522
2,3,24,1,6,0,1.791759,3.970292,2.772589
3,4,24,1,6,0,2.484907,4.663439,2.772589
4,5,24,1,6,0,1.098612,3.496508,2.890372


In [3]:
## Question 2
X = dmtrain.drop(columns = ["response", "id"])
y = dmtrain["response"]

clf = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)

clf.fit(X, y)
clf.score(X, y), clf.score(X, y)

## this is the depth of the tree
depth = clf.get_depth()
print(f"depth is {depth}")

depth is 26


In [4]:
## Question 3
## this is a grid search for the best depth
## chatgpt
from sklearn.model_selection import GridSearchCV


param_grid = {'max_depth': range(1, 20)}
grid_search = GridSearchCV(DecisionTreeClassifier(criterion='entropy'), param_grid, cv=10)
grid_search.fit(X, y)

best_depth = grid_search.best_params_['max_depth']
print("Best tree depth:", best_depth)
print("Accuracy on training data:", grid_search.best_score_)

Best tree depth: 1
Accuracy on training data: 0.708


In [15]:
## Question 4
from sklearn.ensemble import RandomForestClassifier

## Grid search for best depth in random forest
param_grid = {'max_depth': range(1, 20)}
grid_search_rf = GridSearchCV(RandomForestClassifier(n_estimators=100), param_grid, cv=10)
grid_search_rf.fit(X, y)

best_depth_rf = grid_search_rf.best_params_['max_depth']
print("Best tree depth in random forest (100 trees):", best_depth_rf)
print("Accuracy on training data (random forest):", grid_search_rf.best_score_)

Best tree depth in random forest (100 trees): 3
Accuracy on training data (random forest): 0.7085


In [6]:
## Question 5
## Grid search for best depth with 50 trees
grid_search_rf_50 = GridSearchCV(RandomForestClassifier(n_estimators=50), param_grid, cv=10)
grid_search_rf_50.fit(X, y)

best_depth_rf_50 = grid_search_rf_50.best_params_['max_depth']
print("Best tree depth in random forest (50 trees):", best_depth_rf_50)
print("Accuracy on training data (random forest, 50 trees):", grid_search_rf_50.best_score_)

Best tree depth in random forest (50 trees): 4
Accuracy on training data (random forest, 50 trees): 0.7084999999999999


In [7]:
## Question 6

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

## Logistic regression with cross-validation
log_reg = LogisticRegression(max_iter=1000)
accuracy_log_reg = np.mean(cross_val_score(log_reg, X, y, cv=10))

print("Accuracy of logistic regression:", accuracy_log_reg)

Accuracy of logistic regression: 0.7099999999999999


In [8]:
## Question 7
# Based on the accuracy results and the simplicity of the models, I would recommend logistic regression with cross-validation.

# Here's why:

# Accuracy:
# Logistic Regression with cross-validation has an accuracy of 0.71, which is slightly higher than the decision tree and random forest models.
# Model Complexity:
# Logistic Regression is a simple model that tends to perform well with binary classification tasks, like the one you described.
# Decision trees and random forests may have a tendency to overfit, especially when the depth of the trees is not properly tuned, as evidenced by the fact that the best depth for both models is just 1.
# Interpretability:
# Logistic Regression provides easily interpretable coefficients, making it easier to understand the impact of each feature on the prediction.

In [48]:
## Question 8
from sklearn.preprocessing import PowerTransformer

## transform the skewed variables in training data
## perform a power transformation on skewed columns within a DataFrame named 
transformer = PowerTransformer()
# print(dmtrain['transformed_n24'],"TRANSFORMED_N24")
dummy_array=np.array(dmtrain['transformed_n24'])
# dummy_array= dummy_array.reshape(-1, 1)
dummy_array=dummy_array.reshape(-1, 1)
# they transformed the data that is considered to be skewed with the tranformer instancnce 
dmtrain['transformed_n24'] = transformer.fit_transform(dummy_array)

## Read test data
test_data = pd.read_csv("dmtest.csv")
dummy_array2=test_data["rev24"]
dummy_array2=np.array(dummy_array2)
dummy_array2=dummy_array.reshape(-1, 1)
# they put the data back to the test_data coloumn that is already transformed in transfomer. 
test_data["transformed_n24"] = transformer.transform(dummy_array2)


##Separate features and target

#the code bellow is droping the data "response so it not include the fiiting"
X_test = test_data.drop(columns=["response"])
# the data using the response  as a y_test
y_test = test_data["response"]

## Predict using the best model (Random Forest with 100 trees and best depth)
best_model = grid_search_rf.best_estimator_
# predict the towards the best model 
X_test=X_test.drop(columns=["id","transformed_n24"])
y_pred = best_model.predict(X_test)
print(best_model.score)


<bound method ClassifierMixin.score of RandomForestClassifier(max_depth=3)>


In [18]:
# Question 8- other way /// Vee
# Load the stuff we need
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Read data from csv file
df = pd.read_csv("dmtrain.csv")

# We need these columns to predict stuff
features = ['n24', 'rev24', 'revlast', 'elpsdm', 'ordfreq', 'ordcat']
X = df[features]
y = df['response']  # This is what we want to predict

# Split data into parts for training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Make a logistic regression model
model = LogisticRegression(max_iter=1000)  # Use a big number for max_iter to be safe

# Train the model with the training data
model.fit(X_train, y_train)

# Check how well our model works on the test data
y_pred = model.predict(X_test)

# How accurate was our model?
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Get more details on how well our model did
report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)

# Get confusion matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)


Accuracy: 0.6875
Classification Report:
              precision    recall  f1-score   support

           0       0.69      1.00      0.81       272
           1       1.00      0.02      0.05       128

    accuracy                           0.69       400
   macro avg       0.84      0.51      0.43       400
weighted avg       0.79      0.69      0.57       400

Confusion Matrix:
[[272   0]
 [125   3]]
